<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/Copia_di_08_LogReg_100pretr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression on 100 cycles

The file contains 2 version of computation:

2.   Having the real data, consider a part as 1 and another as 0 of the same size
3.   Consider the real data as 1 and create unattested colexifications considered 0

In the case 2, the Regression model tries to predict the most probable colexifications; in the case 3 it tries to predict whether a pair is at all a colexification.

The file outputs two dataframes: df_logreg_bal.csv (balanced) and df_logreg_un.csv (unattested).

NB: it is used the pre-trained version of FastText.

In [ ]:
# these lines print the time of execution in every cell
!pip install ipython-autotime
%load_ext autotime

time: 1.98 ms (started: 2021-04-03 18:01:33 +00:00)


In [ ]:
pip install fasttext

     |████████████████████████████████| 71kB 5.0MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3097120 sha256=9eb9db8f1906cfbc7e39011b195869abcc4aa54833cee7d5b17a8719518a647c
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext
time: 44.4 s (started: 2021-04-03 18:01:34 +00:00)


In [ ]:
import fasttext.util
from google.colab import files
import pickle
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import time
import random

time: 375 ms (started: 2021-04-03 18:02:24 +00:00)


Load the pretrained model

In [ ]:
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

time: 30min 31s (started: 2021-04-03 18:02:30 +00:00)


In [ ]:
import itertools

def cos_sim(a, b):
	"""Takes 2 vectors a, b and returns the cosine similarity according 
	to the definition of the dot product
	"""
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)


# function to write in file
def log(w):
  with open('oov_log.txt', 'a+') as file:
    file.write(w)


# function to handle oov word exception
def trySimilarity(w1, w2, default, model):
  # calculate cosine similarity between two vectors if those exists in the model, else return default --> makes sense? ft deals with OOV
  try:
    v1 = model.get_sentence_vector(w1)
  except:
    log(w1)
    return default
  try:
    v2 = model.get_sentence_vector(w2)
  except:
    log(w2)
    return default
  return cos_sim(v1, v2)


def add_cosine_sim(df,col1,col2,ft_model):
  """
  Enriches dataframe with cosine similarity
  Input: a dataframe, two column names, and a fastText mdodel
  Output: The inputted dataframe with a new column with cosine similarities
  """

  #query the model and if there is an oov word write that word to oov_log.txt and move to the next word

  #add column with cosine sims row-wise

  df['Cosine'] = df.apply(lambda row: trySimilarity(row[col1], row[col2], 0, ft_model), axis=1) 

  return df

time: 28.4 ms (started: 2021-04-03 18:33:34 +00:00)


In [ ]:
!gdown --id 1tu36NAL3MyHMkajE7wEDffbF0OI5fidY #listBoot
# bootstrappping with 100 as number of cicle

Downloading...
From: https://drive.google.com/uc?id=1tu36NAL3MyHMkajE7wEDffbF0OI5fidY
To: /content/listBoot.txt
222MB [00:01, 115MB/s]
time: 5.44 s (started: 2021-04-03 18:33:38 +00:00)


In [ ]:
with open("listBoot.txt", "rb") as fp:   # Unpickling
  lBoot = pickle.load(fp)

time: 9.61 s (started: 2021-04-03 18:33:47 +00:00)


### Version 2

In [ ]:
def timer(start,end):
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)
  print('TIME: '+"{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  print('\n')

def calculate_logreg(nB):  
  
  print ("Running calculate_logreg function")
  start_time = time.time()

  listLogReg = []


  for k in range (nB):
    nBoot = k

    print ("Creating dataframe from bootstrap cicle")
    df = pd.DataFrame(lBoot[nBoot], columns=['Colex_pair','Concept1','Concept2', 'Prob']) # to save in df
    df = df.sort_values(by=['Prob'])
    df = df.reset_index(drop = True)

    # obtain cosine values from model
    print("Obtaining cosine similarity from model")
    add_cosine_sim(df,'Concept1','Concept2', ft)

    # assign 1 to the 2% entries with highest probability
    totN = df.shape[0]
    selected = round(totN * 0.02)
    df ['Colex_bool'] = 0
    df.loc[( totN - selected) : ,'Colex_bool'] = 1

    # select the same number of 0 and 1 pairs --> balance
    g = df.groupby('Colex_bool')
    g = pd.DataFrame(g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True)))

    X = g[['Cosine']]
    y = g['Colex_bool']

    print("Calculating logistic regression scores")
    for i in range(10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
      logreg = LogisticRegression()
      logreg.fit(X_train, y_train)
      y_pred = logreg.predict(X_test)
      f1 = round(f1_score(y_test,y_pred, average='macro'), 2)
      newEl = [nBoot, i, f1] # row: number of boot, random_state, f1 result
      listLogReg.append(newEl)

    print("Computation finished for boot number ", k, '\n')
    timer(start_time, time.time())

  return listLogReg  

time: 45.2 ms (started: 2021-04-03 18:34:16 +00:00)


In [ ]:
random.seed(70)
values = calculate_logreg(len(lBoot))

Running calculate_logreg function
Creating dataframe from bootstrap cicle
Obtaining cosine similarity from model
Calculating logistic regression scores
Computation finished for boot number  0 

TIME: 00:00:02.85


Creating dataframe from bootstrap cicle
Obtaining cosine similarity from model
Calculating logistic regression scores
Computation finished for boot number  1 

TIME: 00:00:05.33


Creating dataframe from bootstrap cicle
Obtaining cosine similarity from model
Calculating logistic regression scores
Computation finished for boot number  2 

TIME: 00:00:08.07


Creating dataframe from bootstrap cicle
Obtaining cosine similarity from model
Calculating logistic regression scores
Computation finished for boot number  3 

TIME: 00:00:10.54


Creating dataframe from bootstrap cicle
Obtaining cosine similarity from model
Calculating logistic regression scores
Computation finished for boot number  4 

TIME: 00:00:13.17


Creating dataframe from bootstrap cicle
Obtaining cosine similarit

In [ ]:
df_logreg_bal = pd.DataFrame(values, columns=['boot_df', 'random_state', 'f1_score'])

In [ ]:
df_logreg_bal.head(2)

,boot_df,random_state,f1_score
0,0,0,0.79
1,0,1,0.79


In [ ]:
df_logreg_bal.to_csv('df_logreg_bal.csv', index = False)

In [ ]:
files.download('df_logreg_bal.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Version 3
Create unattested colexifications ("fake colexifications")

In [ ]:
def calculate_logreg_un(nB):  
  
  print ("Running calculate_logreg_un function")
  start_time = time.time()

  listLogReg_un = []


  for k in range (nB):
    nBoot = k

    print ("Creating dataframe from bootstrap cicle")
    df = pd.DataFrame(lBoot[nBoot], columns=['Colex_pair','Concept1','Concept2', 'Prob']) # to save in df
    df['Colex_bool'] = 1 # all the existing colexifications are real

    real_col = set(list(df['Colex_pair'])) # remove duplicates

    # create list with single items
    real_col_l = [item for t in real_col for item in t]
    real_col_l = list(set(real_col_l)) # delete duplicates

    # create unattested colexifications
    print ("Creating unattested colexifications")
    totN = df.shape[0]
    un_col = []
    for i in range (totN):
      while True:
        el1 = random.choice(real_col_l)
        el2 = random.choice(real_col_l)
        while el1 == el2:
          el2 = random.choice(real_col_l)
        supp_t = (el1, el2)
        if supp_t not in real_col: # fail condition
          un_col.append(supp_t) # append only tuple that is not in real ones
          break

    # create dataframe with 0 values: unattested colexifications
    df_un = pd.DataFrame(un_col, columns =['Concept1', 'Concept2'])
    df_un['Colex_pair'] = un_col
    df_un['Colex_bool'] = 0

    # join two dataframes
    print ("Ultimated dataframe with 0 and 1 values")
    df = df.append(df_un)

    # obtain cosine values from model
    print("Obtaining cosine similarity from model")
    add_cosine_sim(df,'Concept1','Concept2', ft)

    # Logistic Regression
    X = df[['Cosine']]
    y = df['Colex_bool']

    print("Calculating logistic regression scores")
    for i in range(10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
      logreg = LogisticRegression()
      logreg.fit(X_train, y_train)
      y_pred = logreg.predict(X_test)
      f1 = round(f1_score(y_test,y_pred, average='macro'), 2)
      newEl = [nBoot, i, f1] # row: number of boot, random_state, f1 result
      listLogReg_un.append(newEl)

    print("Computation finished for boot number ", k, '\n')
    timer(start_time, time.time())

  return listLogReg_un, df

time: 51.2 ms (started: 2021-04-03 18:39:37 +00:00)


In [ ]:
random.seed(70)
values_un, df_test = calculate_logreg_un(len(lBoot))

Running calculate_logreg_un function
Creating dataframe from bootstrap cicle
Creating unattested colexifications
Ultimated dataframe with 0 and 1 values
Obtaining cosine similarity from model
Calculating logistic regression scores
Computation finished for boot number  0 

TIME: 00:00:06.74


Creating dataframe from bootstrap cicle
Creating unattested colexifications
Ultimated dataframe with 0 and 1 values
Obtaining cosine similarity from model
Calculating logistic regression scores
Computation finished for boot number  1 

TIME: 00:00:12.77


Creating dataframe from bootstrap cicle
Creating unattested colexifications
Ultimated dataframe with 0 and 1 values
Obtaining cosine similarity from model
Calculating logistic regression scores
Computation finished for boot number  2 

TIME: 00:00:19.47


Creating dataframe from bootstrap cicle
Creating unattested colexifications
Ultimated dataframe with 0 and 1 values
Obtaining cosine similarity from model
Calculating logistic regression scores
C

In [ ]:
df_logreg_un = pd.DataFrame(values_un, columns=['boot_df', 'random_state', 'f1_score'])

time: 7.07 ms (started: 2021-04-03 18:59:09 +00:00)


In [ ]:
df_logreg_un.head(2)

,boot_df,random_state,f1_score
0,0,0,0.63
1,0,1,0.63


time: 15.8 ms (started: 2021-04-03 18:59:17 +00:00)


In [ ]:
df_logreg_un.to_csv('df_logreg_un.csv', index = False)

time: 6.48 ms (started: 2021-04-03 18:59:42 +00:00)


In [ ]:
files.download('df_logreg_un.csv')